In [22]:
import nltk
import pandas as pd
import random
pd.set_option('display.max_colwidth',100)
import re
import string
import gensim
#nltk.download()
#python -m nltk.downloader all - from command line
#nltk.download('punkt')
#nltk.download('stopwords')
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer 
from collections import defaultdict
from nltk import pos_tag
from nltk import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from gensim import corpora, models
from gensim.models.nmf import Nmf 
from sklearn.decomposition import NMF

In [ ]:
#Preparing dataset

In [23]:
doc_a = "Brocolli is good to eat. My brother likes to eat good brocolli, but not my mother."
doc_b = "My mother spends a lot of time driving my brother around to baseball practice."
doc_c = "Some health experts suggest that driving may cause increased tension and blood pressure."
doc_d = "I often feel pressure to perform well at school, but my mother never seems to drive my brother to do better."
doc_e = "Health professionals say that brocolli is good for your health."
doc_set = [doc_a, doc_b, doc_c, doc_d, doc_e]

In [ ]:
#Text Preprocessing

In [51]:
# tokenizer = RegexpTokenizer(r'\w+')
# stop_words = stopwords.words('english')
# lemmatizer = WordNetLemmatizer()
# texts = []
# lemmatized_tokens_list=[]
# for i in doc_set:
#     raw = i.lower()
#     tokens = tokenizer.tokenize(raw)
#     stopped_tokens = [i for i in tokens if not i in stop_words]
#     lemmatized_tokens = [lemmatizer.lemmatize(i) for i in stopped_tokens]
#     lemmatized_tokens_list.append(lemmatized_tokens)
#     lemmatized_tokens_combined = ' '.join([lemmatizer.lemmatize(i) for i in stopped_tokens])
#     texts.append(lemmatized_tokens_combined)

# print(texts)
# print(lemmatized_tokens_list)

In [52]:
tokenizer = RegexpTokenizer(r'\w+')
stop_words = stopwords.words('english')
lemmatizer = WordNetLemmatizer()
texts = []
for i in doc_set:
    raw = i.lower()
    tokens = tokenizer.tokenize(raw)
    stopped_tokens = [i for i in tokens if not i in stop_words]
    lemmatized_tokens = [lemmatizer.lemmatize(i) for i in stopped_tokens]
    texts.append(lemmatized_tokens)

print(texts)

[['brocolli', 'good', 'eat', 'brother', 'like', 'eat', 'good', 'brocolli', 'mother'], ['mother', 'spends', 'lot', 'time', 'driving', 'brother', 'around', 'baseball', 'practice'], ['health', 'expert', 'suggest', 'driving', 'may', 'cause', 'increased', 'tension', 'blood', 'pressure'], ['often', 'feel', 'pressure', 'perform', 'well', 'school', 'mother', 'never', 'seems', 'drive', 'brother', 'better'], ['health', 'professional', 'say', 'brocolli', 'good', 'health']]


In [ ]:
#countvectorizer

In [53]:
# #Vectorization
# cv_vect = CountVectorizer(max_df=0.8, min_df=2)
# doc_term_matrix = cv_vect.fit_transform(texts)

# #Model building
# nmf = NMF(n_components=2, random_state=42)
# nmf.fit(doc_term_matrix)

# #fetches 10 words with highest probabilities for all the five topics
# for i,topic in enumerate(nmf.components_):
#     print(f'Top 3 words for topic #{i}:')
#     print([cv_vect.get_feature_names()[i] for i in topic.argsort()[-3:]])

# #saving results
# topic_values = nmf.transform(doc_term_matrix)
# topic_values.shape
# print('\n')
# print('Topics Associated With Documents')
# for i in range(len(doc_set)):
#     print(doc_set[i] + ' - ' + 'Topic ' + str(topic_values.argmax(axis=1)[i]))

In [47]:
#Vectorization
dictionary = corpora.Dictionary(texts)
bow_corpus = [dictionary.doc2bow(text) for text in texts]
print(dictionary)
print(bow_corpus)

#Model building
nmfmodel_bow_corpus = Nmf(bow_corpus, num_topics=2, id2word = dictionary, passes=20)
# corpus (iterable of list of (int, float) or csc_matrix with the shape (n_tokens, n_documents), optional) – Training corpus. 
# Can be either iterable of documents, which are lists of (word_id, word_count) or a sparse csc matrix of BOWs for each document. 
# If not specified, the model is left uninitialized (presumably, to be trained later with self.train()).
# num_topics (int, optional) – Number of topics to extract.
# id2word ({dict of (int, str), gensim.corpora.dictionary.Dictionary}) – Mapping from word IDs to words. 
# It is used to determine the vocabulary size, as well as for debugging and topic printing.
# chunksize (int, optional) – Number of documents to be used in each training chunk.
# passes (int, optional) – Number of full passes over the training corpus. Leave at default passes=1 if your input is an iterator.
# kappa (float, optional) – Gradient descent step size. Larger value makes the model train faster
# but could lead to non-convergence if set too large.
# minimum_probability – If normalize is True, topics with smaller probabilities are filtered out. 
# If normalize is False, topics with smaller factors are filtered out. If set to None, a value of 1e-8 is used to prevent 0s.
# w_max_iter (int, optional) – Maximum number of iterations to train W per each batch.
# w_stop_condition (float, optional) – If error difference gets less than that, training of W stops for the current batch.
# h_max_iter (int, optional) – Maximum number of iterations to train h per each batch.
# h_stop_condition (float) – If error difference gets less than that, training of h stops for the current batch.
# eval_every (int, optional) – Number of batches after which l2 norm of (v - Wh) is computed. Decreases performance if set too low.
# normalize (bool or None, optional) – Whether to normalize the result. Allows for estimation of perplexity, coherence, e.t.c.
# random_state ({np.random.RandomState, int}, optional) – Seed for random generator. Needed for reproducibility.


print(nmfmodel_bow_corpus.print_topics(num_topics=2, num_words=4))

#Evaluating model
coherence_model_lda = models.CoherenceModel(model=nmfmodel_bow_corpus, texts=texts, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

coherence_model_lda = models.CoherenceModel(model=nmfmodel_bow_corpus, texts=texts, dictionary=dictionary, coherence='u_mass')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

Dictionary(33 unique tokens: ['brocolli', 'brother', 'eat', 'good', 'like']...)
[[(0, 2), (1, 1), (2, 2), (3, 2), (4, 1), (5, 1)], [(1, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1)], [(8, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1)], [(1, 1), (5, 1), (19, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1)], [(0, 1), (3, 1), (16, 2), (31, 1), (32, 1)]]
[(0, '0.202*"good" + 0.202*"brocolli" + 0.161*"eat" + 0.084*"mother"'), (1, '0.069*"pressure" + 0.061*"brother" + 0.061*"mother" + 0.059*"driving"')]

Coherence Score:  0.2896196628606057

Coherence Score:  -14.376210806203396


In [43]:
#tfidf

In [45]:
# #Vectorization
# tfidf_vect = TfidfVectorizer(max_df=0.8, min_df=2)
# doc_term_matrix = tfidf_vect.fit_transform(texts)

# #Model building
# nmf = NMF(n_components=2, random_state=42)
# nmf.fit(doc_term_matrix)

# #fetches 10 words with highest probabilities for all the five topics
# for i,topic in enumerate(nmf.components_):
#     print(f'Top 3 words for topic #{i}:')
#     print([tfidf_vect.get_feature_names()[i] for i in topic.argsort()[-3:]])

# #saving results
# topic_values = nmf.transform(doc_term_matrix)
# topic_values.shape
# print('\n')
# print('Topics Associated With Documents')
# for i in range(len(doc_set)):
#     print(doc_set[i] + ' - ' + 'Topic ' + str(topic_values.argmax(axis=1)[i]))

In [48]:
#Vectorization
dictionary = corpora.Dictionary(texts)
bow_corpus = [dictionary.doc2bow(text) for text in texts]
tfidf = models.TfidfModel(bow_corpus)
tfidf_corpus = tfidf[bow_corpus]
print(dictionary)
for doc in tfidf_corpus:
    print(doc)
print(bow_corpus)

#Model building
nmfmodel_tfidf_corpus = Nmf(bow_corpus, num_topics=2, id2word = dictionary, passes=20)
print(nmfmodel_tfidf_corpus.print_topics(num_topics=2, num_words=4))

#Evaluating model
coherence_model_lda = models.CoherenceModel(model=nmfmodel_tfidf_corpus, texts=texts, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

coherence_model_lda = models.CoherenceModel(model=nmfmodel_tfidf_corpus, texts=texts, dictionary=dictionary, coherence='u_mass')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

Dictionary(33 unique tokens: ['brocolli', 'brother', 'eat', 'good', 'like']...)
[(0, 0.40784451109112935), (1, 0.11368521994734913), (2, 0.7163669735975946), (3, 0.40784451109112935), (4, 0.3581834867987973), (5, 0.11368521994734913)]
[(1, 0.12424759593709131), (5, 0.12424759593709131), (6, 0.3914619434234833), (7, 0.3914619434234833), (8, 0.2228684610131362), (9, 0.3914619434234833), (10, 0.3914619434234833), (11, 0.3914619434234833), (12, 0.3914619434234833)]
[(8, 0.2016345105176491), (13, 0.35416512946544426), (14, 0.35416512946544426), (15, 0.35416512946544426), (16, 0.2016345105176491), (17, 0.35416512946544426), (18, 0.35416512946544426), (19, 0.2016345105176491), (20, 0.35416512946544426), (21, 0.35416512946544426)]
[(1, 0.10283764444679584), (5, 0.10283764444679584), (19, 0.18446447498008656), (22, 0.32400646345397865), (23, 0.32400646345397865), (24, 0.32400646345397865), (25, 0.32400646345397865), (26, 0.32400646345397865), (27, 0.32400646345397865), (28, 0.32400646345397865)